In [23]:
from pydantic import BaseModel


class ChatMessage(BaseModel):
    message:str
    response:str = None


class SessionID(BaseModel):
    session_id: str

In [24]:
from schemas import ChatRequest
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()
import os


llm = ChatGroq(model="llama-3.1-8b-instant",
               groq_api_key=os.getenv("GROQ_API_KEY"))

def get_response(state: ChatMessage):

    # USE QUERY
    message = state.message

    # promtp
    prompt = f"Give the user response based on the user query: {message}"

    # llm response
    response_text = llm.invoke(prompt).content

    # update the llm respone in chatRequest model
    state.response = response_text

    return state

In [25]:
query = ChatMessage(message="what is the capital of india?")
response = get_response(state=query)
print(response)

message='what is the capital of india?' response='The capital of India is New Delhi.'


In [140]:
response

ChatMessage(message='what is the capital of india?', response='The capital of India is New Delhi.')

### Add memory

In [100]:
from langchain.memory import ConversationBufferMemory
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from typing import Dict

active_sessions: Dict[str, ConversationBufferMemory] = {}
# memory = ConversationBufferMemory(return_messages=True)

In [87]:
import uuid

def get_or_create_session_id(session_id: str):

    try:
        if session_id not in active_sessions:
            session_id = str(uuid.uuid3().hex)
            return session_id
        else:
            return session_id
    
    except Exception as e:
        print("Session id not found")
        

In [101]:
active_sessions

{}

In [108]:
active_sessions

{}

In [120]:
def get_or_create_session_id(session_id: str | None = None) -> str:

    if session_id not in active_sessions:
        session_id = str(uuid.uuid4().hex)
        active_sessions[session_id] = ConversationBufferMemory()

    else:
        active_sessions[session_id]
    
    return active_sessions[session_id].chat_memory.messages

In [123]:
message_list = get_or_create_session_id(session_id="910525126aa84aa2967f10649f2154c7")
print(message_list)

[]


In [141]:
def update_conversation_database(session_id: str, messages: ChatMessage):


    message_list = get_or_create_session_id(session_id=session_id)
    
    for msg in messages:
        if msg[0] == "message":
            human = HumanMessage(content=msg[1])
            message_list.append(human)

        else:
            ai = AIMessage(content=msg[1])
            message_list.append(ai)

In [144]:
update_conversation_database(session_id="910525126aa84aa2967f10649f2154c7", messages=response)

In [145]:
active_sessions

{'910525126aa84aa2967f10649f2154c7': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='what is the capital of india?', additional_kwargs={}, response_metadata={}), AIMessage(content='The capital of India is New Delhi.', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is the capital of india?', additional_kwargs={}, response_metadata={}), AIMessage(content='The capital of India is New Delhi.', additional_kwargs={}, response_metadata={})])),
 'c5289285661a4116acf3d5b9c34b6ed7': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[('content', 'what is the capital of india?'), ('additional_kwargs', {}), ('response_metadata', {}), ('type', 'human'), ('name', None), ('id', None), ('example', False), ('content', 'The capital of India is New Delhi.'), ('additional_kwargs', {}), ('response_metadata', {}), ('type', 'ai'), ('name', None), ('id', None), ('example', False), ('tool_calls', []), ('invalid_tool_